In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import mrjob as mr


### Homework 5

Copy this notebook. Rename it as: YOURNAME-HW4-mapreduce-XX

with your name replacing YOURNAME and the xx replaced with the date you submit or copy this HW.

Upload your completed jupyter notebook to elearning site as your homework submission. Do not put this notebook on your github.

Do all the homeworks problems below:
As noted doing the homework gets a 3 out of 5. Extension of homework to to implement an TD-IDF algorithm  (see below)

Use the data/bible+shakes.nonpunc.txt file as the source of you analysis in this homework

##Homework 5.1
 A bigram is the combination of words.  Find the 10 most common bigrams from the text.  Order counts in the bigram combination for example "in the" is not the same bigram as "the in"



In [2]:
%%writefile MRWordBigramCount.py
## %load data/bible+shakes.nopunc.txt

# %load code\MRWordFreqCount.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import pandas as pd
import numpy as np

WORD_RE = re.compile(r"[\w']+")


class MRWordBigramCount(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]


    def mapper_get_words(self, _, line):
        lastword=""
        for word in WORD_RE.findall(line):
            if lastword !="" and word !="":
                yield (lastword.lower(), word.lower()), 1
            lastword=word
            
    def combiner_count_words(self, word, counts):
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        #data=pd.DataFrame({'word':word,'count':sum(counts)})
        yield None, (sum(counts), word)
    
    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)


if __name__ == '__main__':
    MRWordBigramCount.run()


Overwriting MRWordBigramCount.py


In [3]:
run MRWordBigramCount.py data/bible+shakes.nopunc.txt

No configs found; falling back on auto-configuration
Creating temp directory c:\users\shard\appdata\local\temp\MRWordBigramCount.shard.20171016.170251.644000
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordBigramCount.shard.20171016.170251.644000\output...


13037	["of", "the"]


Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramCount.shard.20171016.170251.644000...


The highest bigram recorded was "of the", appearing 13037 times. Next I was able to find the top to using stemming.porter2. I got help for this at the following links:

https://pythonhosted.org/mrjob/guides/quickstart.html#running-your-job-different-ways 

https://stackoverflow.com/questions/41763290/top-n-record-mapreduce-on-python 

In [49]:
%%writefile MRWordBigramTop10.py
## %load data/bible+shakes.nopunc.txt

# %load code\MRWordFreqCount.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import pandas as pd
import numpy as np
import re

from stemming.porter2 import stem

WORD_RE = re.compile(r"[\w']+")

class MRWordBigramTop10(MRJob):
    def steps(self):
            return [
                MRStep(mapper=self.mapper,
                    reducer=self.reducer),
                MRStep(reducer = self.secondreducer)
                ]

    def mapper(self,_,line):
        lastword=""
        for word in WORD_RE.findall(line):
            if lastword !="" and word !="":
                newWord=lastword.lower()+" "+word.lower()
                yield (newWord), 1
            lastword=word

    def reducer(self, key, values): 
        yield None, ('%08d'%int(sum(values)),key)

    def secondreducer(self, key, values):   
        self.aList= []    
        for v in values:
            self.aList.append(v)
        count = len(self.aList)
        for m in range(count-10,count):
            yield self.aList[m]


if __name__ == '__main__':
    MRWordBigramTop10.run()

Overwriting MRWordBigramTop10.py


In [50]:
run MRWordBigramTop10.py data/bible+shakes.nopunc.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configu

"00002666"	"i have"
"00002714"	"all the"
"00003013"	"shall be"
"00003020"	"and he"
"00003470"	"i will"
"00003799"	"to the"
"00006738"	"in the"
"00007017"	"the lord"
"00007034"	"and the"
"00013037"	"of the"


Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordBigramTop10.shard.20171016.184550.042000...
Removing temp directory c:\u

The top 10 is:

"00002666"	"i have"

"00002714"	"all the"

"00003013"	"shall be"

"00003020"	"and he"

"00003470"	"i will"

"00003799"	"to the"

"00006738"	"in the"

"00007017"	"the lord"

"00007034"	"and the"

"00013037"	"of the"

## Homework 5.2
Now do the same analysis but make the word order not count "in the" == "the in".  Find the 10 most common ordered bigrams from the alice text.  
 

In [6]:
%%writefile MRWordOrderedBigramCount.py
## %load data/bible+shakes.nopunc.txt

# %load code\MRWordFreqCount.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import pandas as pd
import numpy as np

WORD_RE = re.compile(r"[\w']+")


class MRWordOrderedBigramCount(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]


    def mapper_get_words(self, _, line):
        lastword=""
        for word in WORD_RE.findall(line):
            words=[]
            if lastword !="" and word !="":
                words.append(lastword)
                words.append(word)
                words=sorted(words)
                yield (words[0].lower(), words[1].lower()), 1
            lastword=word
            
    def combiner_count_words(self, word, counts):
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        #data=pd.DataFrame({'word':word,'count':sum(counts)})
        yield None, (sum(counts), word)
    
    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)


if __name__ == '__main__':
    MRWordOrderedBigramCount.run()

Writing MRWordOrderedBigramCount.py


In [7]:
run MRWordOrderedBigramCount.py data/bible+shakes.nopunc.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000
Creating temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000
Creating temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000
Running step 1 of 2...
Running step 1 of 2...
Running step 1 of 2...
Running step 2 of 2...
Running step 2 of 2...
Running step 2 of 2...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000\output...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000\output...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000

13037	["of", "the"]


Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramCount.shard.20171016.171623.114000...


Similarly to the previous, "of the" is the highest ordered bigram, appearing 13037 times. This makes sense that it is the same because "the of" is not used that often.

In [51]:
%%writefile MRWordOrderedBigramTop10.py
## %load data/bible+shakes.nopunc.txt

# %load code\MRWordFreqCount.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import pandas as pd
import numpy as np
import re

from stemming.porter2 import stem

WORD_RE = re.compile(r"[\w']+")

class MRWordOrderedBigramTop10(MRJob):
    def steps(self):
            return [
                MRStep(mapper=self.mapper,
                    reducer=self.reducer),
                MRStep(reducer = self.secondreducer)
                ]

    def mapper(self,_,line):
        lastword=""
        for word in WORD_RE.findall(line):
            words=[]
            if lastword !="" and word !="":
                words.append(lastword)
                words.append(word)
                words=sorted(words)
                newWord=words[0].lower(), words[1].lower()
                yield (newWord), 1
            lastword=word
    
    def reducer(self, key, values): 
        yield None, ('%08d'%int(sum(values)),key)

    def secondreducer(self, key, values):   
        self.aList= []    
        for v in values:
            self.aList.append(v)
        count = len(self.aList)
        for m in range(count-10,count):
            yield self.aList[m]


if __name__ == '__main__':
    MRWordOrderedBigramTop10.run()

Writing MRWordOrderedBigramTop10.py


In [52]:
run MRWordOrderedBigramTop10.py data/bible+shakes.nopunc.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configu

Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000\output...


"00002881"	["am", "i"]
"00003015"	["be", "shall"]
"00003123"	["have", "i"]
"00003129"	["and", "he"]
"00003799"	["the", "to"]
"00004100"	["i", "will"]
"00006739"	["in", "the"]
"00007034"	["and", "the"]
"00007244"	["lord", "the"]
"00013037"	["of", "the"]


Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop10.shard.20171016.185127.514000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedBigramTop

The top 10 ordered bigrams is:

"00002881"	["am", "i"]

"00003015"	["be", "shall"]

"00003123"	["have", "i"]

"00003129"	["and", "he"]

"00003799"	["the", "to"]

"00004100"	["i", "will"]

"00006739"	["in", "the"]

"00007034"	["and", "the"]

"00007244"	["lord", "the"]

"00013037"	["of", "the"]

We can notice a difference between the answer for this and 5.1, because 'shall be' appeared in the first one 3013 times, however now it changed it to 'be shall' which included 3015 times.

## Homework 5.3
A trigram are three word combintation.  Find the 10 most common ordered trigrams from the alice text.  Make it so that the order of the words do not count in the trigram combination for example "in the air" is the same trigram as "the air in" or "air in the"...


In [27]:
%%writefile MRWordTrigramCount.py
## %load data/bible+shakes.nopunc.txt

# %load code\MRWordFreqCount.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import pandas as pd
import numpy as np
import re

WORD_RE = re.compile(r"[\w']+")
data=pd.DataFrame({'word':[],'count':[]})



class MRWordTrigramCount(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        firstword=""
        secondword=""
        for word in WORD_RE.findall(line):
            words=[]
            if firstword !="" and secondword !="" and word!="":
                words.append(firstword)
                words.append(secondword)
                words.append(word)
                words=sorted(words)
                yield (words[0].lower(), words[1].lower(), words[2].lower()), 1
                #yield (firstword.lower(), secondword.lower(), word.lower()), 1
            firstword=secondword
            secondword=word
            
    def combiner_count_words(self, word, counts):
        yield word, sum(counts)

    def reducer_count_words(self, word, counts):
        #data.loc[data.shape[0]]=[word,sum(counts)]
        #yield sorted(zip(word, counts), reverse=True)[:3]
        yield None, (sum(counts), word)
        
        
    def reducer_find_max_word(self, _, word_count_pairs):
        yield max(word_count_pairs)


if __name__ == '__main__':
    MRWordTrigramCount.run()

Overwriting MRWordTrigramCount.py


In [28]:
run MRWordTrigramCount.py data/bible+shakes.nopunc.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000
Creating temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000
Creating temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.

2053	["lord", "of", "the"]


Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordTrigramCount.shard.20171016.174024.007000...
Removing temp direc

The highest trigram was "lord of the" which appeared 2053 times.

In [55]:
%%writefile MRWordOrderedTrigramTop10.py
## %load data/bible+shakes.nopunc.txt

# %load code\MRWordFreqCount.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import pandas as pd
import numpy as np
import re

from stemming.porter2 import stem

WORD_RE = re.compile(r"[\w']+")

class MRWordOrderedTrigramTop10(MRJob):
    def steps(self):
            return [
                MRStep(mapper=self.mapper,
                    reducer=self.reducer),
                MRStep(reducer = self.secondreducer)
                ]

    def mapper(self,_,line):
        firstword=""
        secondword=""
        for word in WORD_RE.findall(line):
            words=[]
            if firstword !="" and secondword !="" and word!="":
                words.append(firstword)
                words.append(secondword)
                words.append(word)
                words=sorted(words)
                newWord=words[0].lower(),words[1].lower(), words[2].lower()
                yield (newWord), 1
                #yield (firstword.lower(), secondword.lower(), word.lower()), 1
            firstword=secondword
            secondword=word
        
    def reducer(self, key, values): 
        yield None, ('%08d'%int(sum(values)),key)

    def secondreducer(self, key, values):   
        self.aList= []    
        for v in values:
            self.aList.append(v)
        count = len(self.aList)
        for m in range(count-10,count):
            yield self.aList[m]


if __name__ == '__main__':
    MRWordOrderedTrigramTop10.run()

Overwriting MRWordOrderedTrigramTop10.py


In [56]:
run MRWordOrderedTrigramTop10.py data/bible+shakes.nopunc.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configu

Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000\output...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000\output...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000\output...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000\output...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000\output...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000\output...
Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000\output...
INFO:mrjob.runner:Streaming final output from c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard

"00000762"	["of", "sons", "the"]
"00000813"	["and", "i", "will"]
"00000855"	["of", "out", "the"]
"00000866"	["lord", "saith", "the"]
"00001025"	["land", "of", "the"]
"00001187"	["and", "lord", "the"]
"00001523"	["of", "son", "the"]
"00001581"	["house", "of", "the"]
"00001794"	["children", "of", "the"]
"00002053"	["lord", "of", "the"]


Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedTrigramTop10.shard.20171016.190054.058000...
Removing temp directory c:\users\shard\appdata\local\temp\MRWordOrderedT

The top 10 ordered trigrams are:

"00000762"	["of", "sons", "the"]

"00000813"	["and", "i", "will"]

"00000855"	["of", "out", "the"]

"00000866"	["lord", "saith", "the"]

"00001025"	["land", "of", "the"]

"00001187"	["and", "lord", "the"]

"00001523"	["of", "son", "the"]

"00001581"	["house", "of", "the"]

"00001794"	["children", "of", "the"]

"00002053"	["lord", "of", "the"]

## Homework 5.4
Create graphs to explain the relationship of the frequency of monograms ( words ) to bigrams and trigam frequencies

My guess for number 4, is that words who have high counts in bigrams, have similarly high counts in trigrams. In other words, since trigrams are composed of bigrams, the bigrams that appear most frequent should be relatively high count in trigrams.

My biggest struggle with this homework was trying to handle the output from MRjob. For instance, I was not able to add the output to a pandas array. I think I would benefit greatly from seeing the solutions to this after all is graded.


## For greater than a score of 3 
###  Create a TD - IDF implementation and 
### Analyze the following Sherlock Holmes book from Project Gutenberg text versions of :
The Adventures of Sherlock Holmes-  http://www.gutenberg.org/ebooks/1661.txt.utf-8

A Study in Scarlet - http://www.gutenberg.org/files/244/244-0.txt

The Hound of the Baskervilles - http://www.gutenberg.org/files/2852/2852-0.txt

The Return of Sherlock Holmes - http://www.gutenberg.org/files/108/108-0.txt

The Sign of the Four - http://www.gutenberg.org/ebooks/2097.txt.utf-8

### Display the scores for the top 20 highest frequencty terms and the relationship to the books

